In [1]:
#### Last update: AS, 11/10/25

In [2]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

## Optimization Code:

### Shared Code:

In [3]:
# General Notes: 0's in arrays represent red integers, 1's in arrays represent blue integers

In [4]:
# Code to generate a random coloring of red and blue in a single array of length n

def generate_array(n):
    return np.random.choice([0, 1], size=n)

In [5]:
# Code to generate a random colorings of red and blue in k arrays of length n

# For kstar: k is number of arms and n is the length of each arm
# For rectangle: k is width of the rectangle and n is the length of the rectangle

def random_kcoloring(k, n):
    coloring = []
    for i in range(k):
        array = generate_array(n)
        coloring += [[array[i] for i in range(n)]]
    return coloring

In [6]:
# Code to generate random ordering of numbers 0 through n-1

def random_ordering(n):
    ordering = list(range(n))
    shuffledordering = random.shuffle(ordering)
    return shuffledordering

### kstar Code:

In [7]:
# General Notes: All coloring arrays begin with the first entry at the end of each arm and the last entry at the center of each arm

##### kstar Counting Code:

In [8]:
# Code to count monochromatic triples within a single arm in kstar

def count_monochromatic_triples(coloring):
    n = len(coloring)
    count = 0

    # Iterate over all possible starting points 'a'
    for a in range(n):
        # Iterate over all possible step sizes 'k'
        k = 1
        while a + 2 * k < n:
            # Check if a, a + k, and a + 2k are the same color
            if coloring[a] == coloring[a + k] == coloring[a + 2 * k]:
                count += 1
            k += 1

    return count

In [9]:
# Code to count monochromatic triples: takes into account all possible parings of arms

def count_kstar(coloring): 
    n = len(coloring[0])
    count = 0

    for arm in coloring:
        count += count_monochromatic_triples(arm)

    for firstarm in range(len(coloring)):
        for secondarm in range(1, len(coloring)):
            tempcoloring = coloring[firstarm].copy() + [coloring[secondarm][n-1-j] for j in range(n)]
            if firstarm < secondarm:
                for i in range(n):
                    current = n + (n-i)%2
                    jrange = []
                    while current < 2*n:
                        jrange = jrange + [current]
                        current += 2
                    for j in jrange:
                        if tempcoloring[i] == tempcoloring[j] and tempcoloring[i] == tempcoloring[int((i + j)/2)]:
                            count += 1
            
    return count

In [10]:
# Code to count monochromatic triples: takes into account only adjacent parings of arms

def count_adjacent_kstar(coloring): 
    n = len(coloring[0])
    count = 0
    
    for arm in coloring:
        count += count_monochromatic_triples(arm)

    for index in range(len(coloring)):
        tempcoloring = coloring[index].copy() + [coloring[(index+1)%len(coloring)][n-1-j] for j in range(n)]
        for i in range(n):
            current = n + (n-i)%2
            jrange = []
            while current < 2*n:
                jrange = jrange + [current]
                current += 2
            for j in jrange:
                if tempcoloring[i] == tempcoloring[j] and tempcoloring[i] == tempcoloring[int((i + j)/2)]:
                    count += 1
            
    return count

##### kstar Swap Code:

In [11]:
# Code to determine if swapping color from red to blue or vise versa decreases number of monomchromatic triples in kstar: takes into account all possible parings of arms

def CheckSwap(coloring, ell):
    n = len(coloring[0])
    numMon = [0,0]
    # h is a helper array:
    h = [i for i in range(n)] + [n - 1 - i for i in range(n)]
    #print(h)

    # First, we decide which arm contains ell:
    OrigArm = coloring[math.floor(1.0*ell/n)]
    EndArms = coloring.copy()
    EndArms.pop(math.floor(1.0*ell/n))

    # Now, we'll define p (pivot) to be the relative position within the arm:
    p = ell%n

    # Now, we start totaling the progressions
    
    # assume position p is the first in the progression
    j = 1
    while p + 2*j < 2*n:
        # All three on the same arm:
        if p + 2*j < n:
            if OrigArm[p + j] == OrigArm[p + 2*j]:
                numMon[OrigArm[p + j]] += 1
        # First two on the first arm:
        elif p + j < n:
            for EndArm in EndArms:
                if OrigArm[p + j] == EndArm[h[p + 2*j]]:
                    numMon[OrigArm[p + j]] += 1
        # Only first one on first arm:
        else:
            for EndArm in EndArms:
                if EndArm[h[p + j]] == EndArm[h[p + 2*j]]:
                    numMon[EndArm[h[p + j]]] += 1
        j += 1

    # now assume p is the second in the progression
    j = 1
    while p + j < 2*n and p - j >= 0:
        # All three on the same arm:
        if p + j < n:
            if OrigArm[p + j] == OrigArm[p - j]:
                numMon[OrigArm[p + j]] += 1
        # First two on original arm:
        else:
            for EndArm in EndArms:
                if OrigArm[p - j] == EndArm[h[p + j]]:
                    numMon[OrigArm[p - j]] += 1
        # Cannot have only one on arm 1.
        j += 1

    # now assume p is third in the progression
    j = 1
    while p - 2*j >= 0:
        if OrigArm[p - j] == OrigArm[p - 2*j]:
            numMon[OrigArm[p-j]] += 1
        j += 1
    
    if numMon[0] < numMon[1]:
        return 0
    # if numMon[0] == numMon[1]:
    #     print("They match.")
    return 1

In [12]:
# Code to determine if swapping color from red to blue or vise versa decreases number of monomchromatic triples in kstar: takes into account only adjacent parings of arms

def CheckSwapAdjacent(coloring, ell):
    n = len(coloring[0])
    k = len(coloring)
    numMon = [0,0]
    # h is a helper array:
    h = [i for i in range(n)] + [n - 1 - i for i in range(n)]
    #print(h)

    # First, we decide which arm contains ell:
    OrigArmIndex = math.floor(1.0*ell/n)
    OrigArm = coloring[OrigArmIndex]
    EndArms = [coloring[(OrigArmIndex-1)%k], coloring[(OrigArmIndex+1)%k]]

    # Now, we'll define p (pivot) to be the relative position within the arm:
    p = ell%n

    # Now, we start totaling the progressions
    
    # assume position p is the first in the progression
    j = 1
    while p + 2*j < 2*n:
        # All three on the same arm:
        if p + 2*j < n:
            if OrigArm[p + j] == OrigArm[p + 2*j]:
                numMon[OrigArm[p + j]] += 1
        # First two on the first arm:
        elif p + j < n:
            for EndArm in EndArms:
                if OrigArm[p + j] == EndArm[h[p + 2*j]]:
                    numMon[OrigArm[p + j]] += 1
        # Only first one on first arm:
        else:
            for EndArm in EndArms:
                if EndArm[h[p + j]] == EndArm[h[p + 2*j]]:
                    numMon[EndArm[h[p + j]]] += 1
        j += 1

    # now assume p is the second in the progression
    j = 1
    while p + j < 2*n and p - j >= 0:
        # All three on the same arm:
        if p + j < n:
            if OrigArm[p + j] == OrigArm[p - j]:
                numMon[OrigArm[p + j]] += 1
        # First two on original arm:
        else:
            for EndArm in EndArms:
                if OrigArm[p - j] == EndArm[h[p + j]]:
                    numMon[OrigArm[p - j]] += 1
        # Cannot have only one on arm 1.
        j += 1

    # now assume p is third in the progression
    j = 1
    while p - 2*j >= 0:
        if OrigArm[p - j] == OrigArm[p - 2*j]:
            numMon[OrigArm[p-j]] += 1
        j += 1
    
    if numMon[0] < numMon[1]:
        return 0
    # if numMon[0] == numMon[1]:
    #     print("They match.")
    return 1

##### kstar Optimization Code:

In [13]:
# Code that determines optimal coloring to minimize monochromatic triples in kstar: takes into account only adjacent parings of arms

def optimize_adjacent_kstar_coloring(coloring, ordering=None):
    n = len(coloring[0])
    k = len(coloring)
    currentcoloring = coloring

    if ordering == None:
        ordering = [i for i in range(k*n)]

    diditchange = 0
    while diditchange == 0:
        diditchange = 1
        for j in ordering:
            kcurrent = currentcoloring[math.floor(j/n)][j%n]
            knew = CheckSwapAdjacent(currentcoloring, j)
            if kcurrent != knew:
                diditchange = 0
                currentcoloring[math.floor(j/n)][j%n] = knew
                
    print("Performance: ", count_adjacent_kstar(currentcoloring))
    plot_kstar_coloring(currentcoloring)

    return currentcoloring

In [14]:
# Code that determines optimal coloring to minimize monochromatic triples in kstar: takes into account all possible parings of arms

def optimize_kstar_coloring(coloring, ordering=None):
    n = len(coloring[0])
    k = len(coloring)
    currentcoloring = coloring

    if ordering == None:
        ordering = [i for i in range(k*n)]

    diditchange = 0
    while diditchange == 0:
        diditchange = 1
        for j in ordering:
            kcurrent = currentcoloring[math.floor(j/n)][j%n]
            knew = CheckSwap(currentcoloring, j)
            if kcurrent != knew:
                diditchange = 0
                currentcoloring[math.floor(j/n)][j%n] = knew
                
    print("Performance: ", count_kstar(currentcoloring))
    plot_kstar_coloring(currentcoloring)

    return currentcoloring

##### kstar Plotting Code:

In [15]:
# # Code for plotting kstar

# def plot_kstar_coloring(currentcoloring, filename=None):
#     n = len(currentcoloring[0])
#     k = len(currentcoloring)

#     plt.figure(figsize=(4, 4))
    
#     plt.xticks([])
#     plt.yticks([])

#     # Plot the center of the picture first

#     for i in range(k):
#         x_points = [(1)*math.cos((2*math.pi*i)/k), (0)*math.cos((2*math.pi*i)/k)]
#         y_points = [(1)*math.sin((2*math.pi*i)/k), (0)*math.sin((2*math.pi*i)/k)]
#         if currentcoloring[i][n-1] == 0:
#             plt.plot(x_points, y_points, linestyle='-', color='red', linewidth='5')
#         else:
#             plt.plot(x_points, y_points, linestyle='-', color='blue', linewidth='5')
    

#     # Plot the rest of the picture
    
#     for i in range(k):

#         if currentcoloring[i][0] == 0 and currentcoloring[i][1] == 0:
#             regime = 'red'
#         elif currentcoloring[i][0] == 1 and currentcoloring[i][1] == 1:
#             regime = 'blue'
#         else:
#             regime = 'purple'

#         regimebroken = False

#         count = 2
        
#         for j in range(2,n):

#             if regime == 'red' and currentcoloring[i][j] == 1:
#                 regimebroken = True

#             if regime == 'blue' and currentcoloring[i][j] == 0:
#                 regimebroken = True

#             if regime == 'purple' and currentcoloring[i][j] == currentcoloring[i][j-1]:
#                 regimebroken = True

#             if regimebroken == False:
#                 count += 1

#             if regimebroken == True or j == n-1:
#                 x_points = [(n-j)*math.cos((2*math.pi*i)/k), (n-(j-count))*math.cos((2*math.pi*i)/k)]
#                 y_points = [(n-j)*math.sin((2*math.pi*i)/k), (n-(j-count))*math.sin((2*math.pi*i)/k)]
#                 if regime == 'red':
#                     plt.plot(x_points, y_points, linestyle='-', color='red', linewidth='5')
#                 elif regime == 'blue':
#                     plt.plot(x_points, y_points, linestyle='-', color='blue', linewidth='5')
#                 else:
#                     plt.plot(x_points, y_points, linestyle='-', color='darkorchid', linewidth='5')

#                 regimebroken = False
#                 count = 1

#                 if j < n-1:
#                     if currentcoloring[i][j] == 0 and currentcoloring[i][j+1] == 0:
#                         regime = 'red'
#                     elif currentcoloring[i][j] == 1 and currentcoloring[i][j+1] == 1:
#                         regime = 'blue'
#                     else:
#                         regime = 'purple'

#     if filename:
#         plt.savefig(filename, bbox_inches="tight")
    
#     plt.show()

In [16]:
# Code for plotting kstar

def plot_kstar_coloring(currentcoloring, filename=None):
    n = len(currentcoloring[0])
    k = len(currentcoloring)

    plt.figure(figsize=(4, 4))
    
    plt.xticks([])
    plt.yticks([])

    # Plot the center of the picture first

    for i in range(k):
        x_points = [(1)*math.cos((2*math.pi*i)/k), (0)*math.cos((2*math.pi*i)/k)]
        y_points = [(1)*math.sin((2*math.pi*i)/k), (0)*math.sin((2*math.pi*i)/k)]
        if currentcoloring[i][n-1] == 0:
            plt.plot(x_points, y_points, linestyle='-', color='red', linewidth='5')
        else:
            plt.plot(x_points, y_points, linestyle='-', color='blue', linewidth='5')
    

    # Plot the rest of the picture
    
    for i in range(k):

        if currentcoloring[i][0] == 0 and currentcoloring[i][1] == 0:
            regime = 'red'
        elif currentcoloring[i][0] == 1 and currentcoloring[i][1] == 1:
            regime = 'blue'
        elif (n%2 == 0 and currentcoloring[i][0] == 1) or (n%2 == 1 and currentcoloring[i][0] == 0):
            regime = 'purple'
        else:
            regime = 'pink'

        regimebroken = False

        count = 2
        
        for j in range(2,n):

            if regime == 'red' and currentcoloring[i][j] == 1:
                regimebroken = True

            if regime == 'blue' and currentcoloring[i][j] == 0:
                regimebroken = True

            if (regime == 'purple' or regime == 'pink') and currentcoloring[i][j] == currentcoloring[i][j-1]:
                regimebroken = True

            if regimebroken == False:
                count += 1

            if regimebroken == True or j == n-1:
                x_points = [(n-j)*math.cos((2*math.pi*i)/k), (n-(j-count))*math.cos((2*math.pi*i)/k)]
                y_points = [(n-j)*math.sin((2*math.pi*i)/k), (n-(j-count))*math.sin((2*math.pi*i)/k)]
                if regime == 'red':
                    plt.plot(x_points, y_points, linestyle='-', color='red', linewidth='5')
                elif regime == 'blue':
                    plt.plot(x_points, y_points, linestyle='-', color='blue', linewidth='5')
                elif regime == 'purple':
                    plt.plot(x_points, y_points, linestyle='-', color='darkorchid', linewidth='5')
                    # plt.plot(x_points, y_points, linestyle='-', color='yellow', linewidth='5')
                else:
                    plt.plot(x_points, y_points, linestyle='-', color='fuchsia', linewidth='5')

                regimebroken = False
                count = 1

                if j < n-1:
                    if currentcoloring[i][j] == 0 and currentcoloring[i][j+1] == 0:
                        regime = 'red'
                    elif currentcoloring[i][j] == 1 and currentcoloring[i][j+1] == 1:
                        regime = 'blue'
                    elif (currentcoloring[i][j] == 1 and (n - i)%2 == 0) or (currentcoloring[i][j] == 0 and (n - i)%2 == 1):
                        regime = 'purple'
                    else:
                        regime = 'pink'

    if filename:
        plt.savefig(filename, bbox_inches="tight")
    plt.show()

In [ ]:
plot_kstar_coloring(currentcoloring, filename='ExampleColoring.pdf')

### Rectangle Code

##### Rectangle Counting Code:

In [16]:
# Code to count monochromatic progressions within rectangles: assumes up/down OR left/right progressions

def count_monochromatic_triples_rectangle_UDLR(coloring):
    k = len(coloring)
    n = len(coloring[0])
    count = 0
    # We'll count only left-to-right and up-to-down progressions
    for i in range(n):
        for j in range(k):
            dL = 1
            while i + 2*dL < n:
                if coloring[j][i] == coloring[j][i + dL] and coloring[j][i] == coloring[j][i + 2*dL]:
                    count += 1
                dL += 1
            dU = 1
            while j + 2*dU < k:
                if coloring[j][i] == coloring[j + dU][i] and coloring[j][i] == coloring[j + 2*dU][i]:
                    count += 1
                dU += 1
    return count

In [17]:
# Code to count monochromatic progressions within rectangles: assumes horizontal and vertical directions are either both positive or both negative

def count_monochromatic_triples_rectangle_posdiag(coloring):
    k = len(coloring)
    n = len(coloring[0])
    count = 0
    for i in range(n):
        for j in range(k):
            dL = 0
            while i + 2*dL < n:
                dU = 0
                while j + 2*dU < k:
                    if coloring[j][i] == coloring[j + dU][i + dL] and coloring[j][i] == coloring[j + 2*dU][i + 2*dL] and (dL != 0 or dU != 0):
                        count += 1
                    dU += 1
                dL += 1
    return count

In [18]:
# Code to count monochromatic progressions within rectangles: includes all diagonal directions

def count_monochromatic_triples_rectangle_diag(coloring):
    k = len(coloring)
    n = len(coloring[0])
    count = 0
    for i in range(n):
        for j in range(k):
            # both positive
            dL = 0
            while i + 2*dL < n:
                dU = 0
                while j + 2*dU < k:
                    if coloring[j][i] == coloring[j + dU][i + dL] and coloring[j][i] == coloring[j + 2*dU][i + 2*dL] and (dL != 0 or dU != 0):
                        count += 1
                    dU += 1
                dL += 1

            # one positive, one negative
            # this part was added 4/12/2025
            # note that we only need one to be negative.  That's because flipping the direction in both axes gives all the opposite progressions.
            # we don't want to add back in cases where dU or dL = 0, though
            dL = 0
            while i + 2*dL < n:
                dU = 0
                while j - 2*dU >= 0:
                    if coloring[j][i] == coloring[j - dU][i + dL] and coloring[j][i] == coloring[j - 2*dU][i + 2*dL] and (dL != 0 and dU != 0):
                        count += 1
                    dU += 1
                dL += 1
            
    return count

##### Rectangle Swap Code:

In [19]:
# Code to determine if swapping color from red to blue or vise versa decreases number of monomchromatic triples in rectangle: assumes up/down OR left/right progressions

def CheckSwap_Rectangle_UDLR(coloring, j, i, numColors = 2):
    k = len(coloring)
    n = len(coloring[0])
    numMon = [0]*numColors

    # Instead of nested while loops, we'll now have a horizontal while loop and a vertical while loop for each case.
    
    # First, we'll assume (i, j) is the first term in the progression:
    dL = 1
    while i + 2*dL < n:
        if coloring[j][i + dL] == coloring[j][i + 2*dL]:
                numMon[coloring[j][i + dL]] += 1
        dL += 1
        
    dU = 1
    while j + 2*dU < k:
        if coloring[j + dU][i] == coloring[j + 2*dU][i]:
            numMon[coloring[j + dU][i]] += 1
        dU += 1

    # (i, j) is the second term:
    dL = 1
    while i - dL >= 0 and i + dL < n:
        if coloring[j][i + dL] == coloring[j][i - dL]:
                numMon[coloring[j][i + dL]] += 1
        dL += 1
        
    dU = 1
    while j - dU >= 0 and j + dU < k:
        if coloring[j + dU][i] == coloring[j - dU][i]:
            numMon[coloring[j + dU][i]] += 1
        dU += 1

    # (i, j) is the third term:
    dL = 1
    while i - 2*dL >= 0:
        if coloring[j][i - 2*dL] == coloring[j][i - dL]:
            numMon[coloring[j][i - dL]] += 1
        dL += 1
    
    dU = 1
    while j - 2*dU >= 0:
        if coloring[j - 2*dU][i] == coloring[j - dU][i]:
            numMon[coloring[j - dU][i]] += 1
        dU += 1

    min = numMon[0]
    minindex = 0
    index = 1

    while index < numColors:
        if numMon[index] < min:
            min = numMon[index]
            minindex = index
        index += 1

    return minindex

In [20]:
# Code to determine if swapping color from red to blue or vise versa decreases number of monomchromatic triples in rectangle: assumes horizontal and vertical directions are either both positive or both negative

def CheckSwap_Rectangle_PosDiag(coloring, j, i, numColors = 2):
    ### Caution!  The case where dL = 0 or dU = 0 is annoying.
    k = len(coloring)
    n = len(coloring[0])
    numMon = [0]*numColors

    # First, we'll assume (i, j) is the first term in the progression:
    dL = 0
    while i + 2*dL < n:
        dU = 0
        while j + 2*dU < k:
            if coloring[j + dU][i + dL] == coloring[j + 2*dU][i + 2*dL] and (dU != 0 or dL != 0):
                numMon[coloring[j + dU][i + dL]] += 1
            dU += 1
        dL += 1

    # (i, j) is the second term:
    dL = 0
    while i - dL >= 0 and i + dL < n:
        dU = 0
        while j - dU >= 0 and j + dU < k:
            if coloring[j + dU][i + dL] == coloring[j - dU][i - dL] and (dU != 0 or dL != 0):
                numMon[coloring[j + dU][i + dL]] += 1
            dU += 1
        dL += 1

    # (i, j) is the third term:
    dL = 0
    while i - 2*dL >= 0:
        dU = 0
        while j - 2*dU >= 0:
            if coloring[j - 2*dU][i - 2*dL] == coloring[j - dU][i - dL] and (dU != 0 or dL != 0):
                numMon[coloring[j - dU][i - dL]] += 1
            dU += 1
        dL += 1

    min = numMon[0]
    minindex = 0
    index = 1

    while index < numColors:
        if numMon[index] < min:
            min = numMon[index]
            minindex = index
        index += 1

    return minindex

In [21]:
# Code to determine if swapping color from red to blue or vise versa decreases number of monomchromatic triples in rectangle: includes all diagonal directions

def CheckSwap_Rectangle_Diag(coloring, j, i, numColors = 2):
    ### Caution!  The case where dL = 0 or dU = 0 is annoying.
    k = len(coloring)
    n = len(coloring[0])
    numMon = [0]*numColors

    # First, we'll assume (i, j) is the first term in the progression:
    # matching directions
    dL = 0
    while i + 2*dL < n:
        dU = 0
        while j + 2*dU < k:
            if coloring[j + dU][i + dL] == coloring[j + 2*dU][i + 2*dL] and (dU != 0 or dL != 0):
                numMon[coloring[j + dU][i + dL]] += 1
            dU += 1
        dL += 1

    # mismatched directions
    dL = 0
    while i + 2*dL < n:
        dU = 0
        while j - 2*dU >= 0:
            if coloring[j - dU][i + dL] == coloring[j - 2*dU][i + 2*dL] and (dU != 0 and dL != 0):
                numMon[coloring[j - dU][i + dL]] += 1
            dU += 1
        dL += 1

    # (i, j) is the second term:
    # matching directions
    dL = 0
    while i - dL >= 0 and i + dL < n:
        dU = 0
        while j - dU >= 0 and j + dU < k:
            if coloring[j + dU][i + dL] == coloring[j - dU][i - dL] and (dU != 0 or dL != 0):
                numMon[coloring[j + dU][i + dL]] += 1
            dU += 1
        dL += 1

    # mismatched directions
    dL = 0
    while i - dL >= 0 and i + dL < n:
        dU = 0
        while j - dU >= 0 and j + dU < k:
            if coloring[j - dU][i + dL] == coloring[j + dU][i - dL] and (dU != 0 and dL != 0):
                numMon[coloring[j - dU][i + dL]] += 1
            dU += 1
        dL += 1

    # (i, j) is the third term:
    # matching directions
    dL = 0
    while i - 2*dL >= 0:
        dU = 0
        while j - 2*dU >= 0:
            if coloring[j - 2*dU][i - 2*dL] == coloring[j - dU][i - dL] and (dU != 0 or dL != 0):
                numMon[coloring[j - dU][i - dL]] += 1
            dU += 1
        dL += 1

    # mismatched
    dL = 0
    while i - 2*dL >= 0:
        dU = 0
        while j + 2*dU < k:
            if coloring[j + 2*dU][i - 2*dL] == coloring[j + dU][i - dL] and (dU != 0 and dL != 0):
                numMon[coloring[j + dU][i - dL]] += 1
            dU += 1
        dL += 1

    min = numMon[0]
    minindex = 0
    index = 1

    while index < numColors:
        if numMon[index] < min:
            min = numMon[index]
            minindex = index
        index += 1

    return minindex

##### Rectangle Optimization Code:

In [22]:
# Code that determines optimal coloring to minimize monochromatic triples in rectangle

# This code has a couple flags:
# First, progressiontype can be:
# 'D' if all diagonals are allowed,
# 'P' if only positive diagonals are allowed
# 'UD' if only up/down or left/right progressions are allowed
# 
# Then, aspect can be:
# True for square pictures
# False for rectangular pictures

def optimize_rectangle_coloring(coloring, progressiontype = 'D', ordering = None, aspect = True):

    n = len(coloring[0])
    k = len(coloring)
    
    diditchange = 0

    currentcoloring = coloring

    if ordering==None:
        ordering=[i for i in range(k*n)]

    if progressiontype == 'D':
        while diditchange == 0:
            diditchange = 1
            for l in ordering:
                kcurrent = currentcoloring[math.floor(l/n)][l%n]
                knew = CheckSwap_Rectangle_Diag(currentcoloring, math.floor(l/n), l%n)
                if kcurrent != knew:
                    diditchange = 0
                    currentcoloring[math.floor(l/n)][l%n] = knew

        print("Performance: ", count_monochromatic_triples_rectangle_diag(currentcoloring))
        plot_rectangle_coloring(currentcoloring, equal=aspect)


    
    elif progressiontype == 'P':
        while diditchange == 0:
            diditchange = 1
            for l in ordering:
                kcurrent = currentcoloring[math.floor(l/n)][l%n]
                knew = CheckSwap_Rectangle_PosDiag(currentcoloring, math.floor(l/n), l%n)
                if kcurrent != knew:
                    diditchange = 0
                    currentcoloring[math.floor(l/n)][l%n] = knew

        print("Performance: ", count_monochromatic_triples_rectangle_posdiag(currentcoloring))
        plot_rectangle_coloring(currentcoloring, equal=aspect) 


    
    elif progressiontype == 'UD':
        while diditchange == 0:
            diditchange = 1
            for l in ordering:
                kcurrent = currentcoloring[math.floor(l/n)][l%n]
                knew = CheckSwap_Rectangle_UDLR(currentcoloring, math.floor(l/n), l%n)
                if kcurrent != knew:
                    diditchange = 0
                    currentcoloring[math.floor(l/n)][l%n] = knew

        print("Performance: ", count_monochromatic_triples_rectangle_UDLR(currentcoloring))
        plot_rectangle_coloring(currentcoloring, equal=aspect) 

    
    else:
        print("Progression type not an option.")
        return
    return currentcoloring

##### Rectangle Plotting Code:

In [23]:
# Code to plot rectangle
# From ChatGPT

def plot_rectangle_coloring(Coloring, equal=True):
    rows, cols = len(Coloring), len(Coloring[0])
    fig, ax = plt.subplots()
    ax.set_xticks(np.arange(cols + 1), minor=False)
    ax.set_yticks(np.arange(rows + 1), minor=False)
    # ax.grid(which="minor", color="black", linestyle='-', linewidth=2)
    ax.tick_params(which="both", bottom=False, left=False, labelbottom=False, labelleft=False)
    
    for i in range(rows):
        for j in range(cols):
            color = 'blue' if Coloring[i][j] == 0 else 'red'
            ax.add_patch(plt.Rectangle((j, i), 1, 1, color=color))
    
    ax.set_xlim(0, cols)
    ax.set_ylim(0, rows)
    if equal:
        ax.set_aspect('equal')
    plt.gca().invert_yaxis()
    plt.show()

### Circle Code:

In [24]:
# General Notes: We still assume a circular coloring is stored as an array of arrays, but we fill the parts that don't correspond to the circle with -1.

##### Circle Generation Code:

In [25]:
# Code to generate circle colorings

def random_circle(n):
    coloring = []
    for j in range(2*n+1):
        newarray = generate_array(2*n+1)
        coloring += [[newarray[i] for i in range(2*n+1)]]
    for i in range(2*n + 1):
        for j in range(2*n + 1):
            #print((i-n)**2 + (j-n)**2)
            if (i-n)**2 + (j-n)**2 > n**2:
                coloring[i][j] = -1
    return coloring

In [26]:
# Code to generate circle colorings

def all_one_circle(n):
    coloring = []
    for j in range(2*n+1):
        coloring += [[1 for i in range(2*n+1)]]
    for i in range(2*n + 1):
        for j in range(2*n + 1):
            #print((i-n)**2 + (j-n)**2)
            if (i-n)**2 + (j-n)**2 > n**2:
                coloring[i][j] = -1
    return coloring

##### Circle Count Code:

In [27]:
# Code to count monochromatic progressions within circles.

# This code has a flag:
# progressiontype can be:
# 'D' if all diagonals are allowed,
# 'P' if only positive diagonals are allowed

def count_monochromatic_circle(coloring, progressiontype='D'):
    n = int(1.0*(len(coloring)-1)/2)
    # print(n)
    count = 0
    
    if progressiontype not in ['P', 'D', 'UD']:
        print("The progression type is not an allowable type.  The choices are:")
        print("P: positive diagonals")
        print("D: all diagonals")
        print("UD: up/down and left/right progressions only")
        return
    
    if progressiontype=='D' or progressiontype=='P':
        for i in range(2*n+1):
            for j in range(2*n+1):
                if (i-n)**2 + (j-n)**2 <= n**2:
                    # print("(j, i): (", j, ", ", i, ")")
                    # both positive
                    dL = 0
                    # This is a bit complicated, but I think i + 2dL - n may not appear with j - n and hence is not the correct bound for dL.
                    # while (i + 2*dL - n)**2 + (j - n)**2 <= n**2:
                    while (i + 2*dL - n)**2 <= n**2:
                        dU = 0
                        # while (i + 2*dL - n)**2 + (j + 2*dU - n)**2 <= n**2:
                        while (j + 2*dU - n)**2 <= n**2:
                            if (i + 2*dL - n)**2 + (j + 2*dU - n)**2 <= n**2:
                                if coloring[j][i] == coloring[j + dU][i + dL] and coloring[j][i] == coloring[j + 2*dU][i + 2*dL] and (dL != 0 or dU != 0):
                                    count += 1
                            dU += 1
                        dL += 1
                    
                    dL = 0
    
                    if progressiontype=='D':
                        # while (i + 2*dL - n)**2 + (j - n)**2 <= n**2:
                        while (i + 2*dL - n)**2 <= n**2:
                            dU = 0
                            while (j - 2*dU - n)**2 <= n**2:
                                if (i + 2*dL - n)**2 + (j - 2*dU - n)**2 <= n**2:
                                    if coloring[j][i] == coloring[j - dU][i + dL] and coloring[j][i] == coloring[j - 2*dU][i + 2*dL] and (dL != 0 and dU != 0):
                                        count += 1
                                dU += 1
                            dL += 1
        
    if progressiontype == 'UD':
        for i in range(2*n+1):
            for j in range(2*n+1):
                if (i-n)**2 + (j-n)**2 <= n**2:
                    # print("(j, i): (", j, ", ", i, ")")
                    # both positive
                    dL = 1
                    while (i + 2*dL - n)**2 <= n**2:
                        if (i + 2*dL - n)**2 + (j-n)**2 <= n**2:
                            if coloring[j][i] == coloring[j][i + dL] and coloring[j][i] == coloring[j][i + 2*dL]:
                                count += 1
                        dL += 1

                    dU = 1
                    while (j + 2*dU - n)**2 <= n**2:
                        if (i-n)**2 + (j + 2*dU - n) <= n**2:
                            if coloring[j][i] == coloring[j + dU][i] and coloring[j][i] == coloring[j + 2*dU][i]:
                                count += 1
                        dU += 1

        
            
    return count

##### Circle Swap Code:

In [28]:
# Code to determine if swapping color from red to blue or vise versa decreases number of monomchromatic triples in circle

# This code has a flag:
# progressiontype can be:
# 'D' if all diagonals are allowed,
# 'P' if only positive diagonals are allowed
# 'UD' if only up/down and left/right progressions are allowed

def CheckSwap_Circle(coloring, j, i, numColors = 2, progressiontype='D', verbose=False):
    ### Caution!  The case where dL = 0 or dU = 0 is annoying.
    
    if progressiontype not in ['P', 'D', 'UD']:
        print("The progression type is not an allowable type.  The choices are:")
        print("P: positive diagonals")
        print("D: all diagonals")
        print("UD: up/down and left/right progressions only")
        return
    
    n = int(1.0*(len(coloring)-1)/2)

    if (i-n)**2 + (j-n)**2 > n**2:
        print("error: change point not within the circle.")
        return -1
    
    numMon = [0]*numColors

    # First, we'll assume (j, i) is the first term in the progression:
    # matching directions
    if progressiontype == 'D' or progressiontype == 'P':
        dL = 0
        while (i + 2*dL - n)**2 <= n**2:
        # while i + 2*dL < n:
            dU = 0
            # while j + 2*dU < k:
            while (j + 2*dU - n)**2 <= n**2:
                if (i + 2*dL - n)**2 + (j + 2*dU - n)**2 <= n**2:
                    # This block counts UD/LR progressions for 'D' and 'P' case
                    if coloring[j + dU][i + dL] == coloring[j + 2*dU][i + 2*dL] and (dU != 0 or dL != 0):
                        if coloring[j + dU][i + dL] == 1 and verbose == True:
                            print("(", i, ", ", j, "), (", i + dL, ", ", j + dU, "), (", i + 2*dL, ", ", j + 2*dU, ")")
                        numMon[coloring[j + dU][i + dL]] += 1
                dU += 1
            dL += 1
        
    # mismatched directions
    if progressiontype == 'D':
        dL = 0
        while (i + 2*dL - n)**2 <= n**2:
        # while i + 2*dL < n:
            dU = 0
            # while j - 2*dU >= 0:
            while (j - 2*dU - n)**2 <= n**2:
                if (i + 2*dL - n)**2 + (j - 2*dU - n)**2 <= n**2:
                    # We don't want to count UD/LR progressions again, so now "dU != 0 AND dL != 0")
                    if coloring[j - dU][i + dL] == coloring[j - 2*dU][i + 2*dL] and (dU != 0 and dL != 0):
                        if coloring[j - dU][i + dL] == 1 and verbose == True:
                            print("(", i, ", ", j, "), (", i + dL, ", ", j - dU, "), (", i + 2*dL, ", ", j - 2*dU, ")")
                        numMon[coloring[j - dU][i + dL]] += 1
                dU += 1
            dL += 1

    # (i, j) is the second term:
    # matching directions
    if progressiontype == 'D' or progressiontype == 'P':
        dL = 0
        while (i - dL - n)**2 <= n**2 and (i + dL - n)**2 <= n**2:
        # while i - dL >= 0 and i + dL < n:
            dU = 0
            # while j - dU >= 0 and j + dU < k:
            while (j - dU - n)**2 <= n**2 and (j + dU - n)**2 <= n**2:
                if (i - dL - n)**2 + (j - dU - n)**2 <= n**2 and (i + dL - n)**2 + (j + dU - n)**2 <= n**2:
                    if coloring[j + dU][i + dL] == coloring[j - dU][i - dL] and (dU != 0 or dL != 0):
                        if coloring[j + dU][i + dL] == 1 and verbose == True:
                            print("(", i - dL, ", ", j - dU, "), (", i, ", ", j, "), (", i + dL, ", ", j + dU, ")")
                        numMon[coloring[j + dU][i + dL]] += 1
                dU += 1
            dL += 1

    # mismatched directions
    if progressiontype == 'D':
        dL = 0
        while (i - dL - n)**2 <= n**2 and (i + dL - n)**2 <= n**2:
        # while i - dL >= 0 and i + dL < n:
            dU = 0
            # while j - dU >= 0 and j + dU < k:
            while (j - dU - n)**2 <= n**2 and (j + dU - n)**2 <= n**2:
                if (i + dL - n)**2 + (j - dU - n)**2 <= n**2 and (i - dL - n)**2 + (j + dU - n)**2 <= n**2:
                    if coloring[j - dU][i + dL] == coloring[j + dU][i - dL] and (dU != 0 and dL != 0):
                        if coloring[j - dU][i + dL] == 1 and verbose == True:
                            print("(", i + dL, ", ", j - dU, "), (", i, ", ", j, "), (", i - dL, ", ", j + dU, ")")
                        numMon[coloring[j - dU][i + dL]] += 1
                dU += 1
            dL += 1

    # (i, j) is the third term:
    # matching directions
    if progressiontype == 'D' or progressiontype == 'P':
        dL = 0
        # while i - 2*dL >= 0:
        while (i - 2*dL - n)**2 <= n**2:
            dU = 0
            # while j - 2*dU >= 0:
            while (j - 2*dU - n)**2 <= n**2:
                if (i - 2*dL - n)**2 + (j - 2*dU - n)**2 <= n**2:
                    if coloring[j - 2*dU][i - 2*dL] == coloring[j - dU][i - dL] and (dU != 0 or dL != 0):
                        if coloring[j - dU][i - dL] == 1 and verbose == True:
                            print("(", i - 2*dL, ", ", j - 2*dU, "), (", i - dL, ", ", j - dU, "), (", i, ", ", j, ")")
                        numMon[coloring[j - dU][i - dL]] += 1
                dU += 1
            dL += 1

    # mismatched directions
    if progressiontype == 'D':
        dL = 0
        while (i - 2*dL - n)**2 <= n**2:
        # while i - 2*dL >= 0:
            dU = 0
            # while j + 2*dU < k:
            while (j + 2*dU - n)**2 <= n**2:
                if (i - 2*dL - n)**2 + (j + 2*dU - n)**2 <= n**2:
                    if coloring[j + 2*dU][i - 2*dL] == coloring[j + dU][i - dL] and (dU != 0 and dL != 0):
                        if coloring[j - dU][i - dL] == 1 and verbose == True:
                            print("(", i - 2*dL, ", ", j + 2*dU, "), (", i - dL, ", ", j + dU, "), (", i, ", ", j, ")")
                        numMon[coloring[j + dU][i - dL]] += 1
                dU += 1
            dL += 1

    if progressiontype == 'UD':
        #First term:
        dL = 1
        while (i + 2*dL - n)**2 + (j - n)**2 <= n**2:
            if coloring[j][i + dL] == coloring[j][i + 2*dL]:
                numMon[coloring[j][i+dL]] += 1
            dL += 1
        dU = 1
        while (i - n)**2 + (j + 2*dU - n)**2 <= n**2:
            if coloring[j + dU][i] == coloring[j + 2*dU][i]:
                numMon[coloring[j + dU][i]] += 1
            dU += 1
        #Second term:
        dL = 1
        while (i + dL - n)**2 + (j - n)**2 <= n**2 and (i - dL - n)**2 + (j - n)**2 <= n**2:
            if coloring[j][i + dL] == coloring[j][i - dL]:
                numMon[coloring[j][i + dL]] += 1
            dL += 1
        dU = 1
        while (i - n)**2 + (j + dU - n)**2 <= n**2 and (i - n)**2 + (j - dU - n)**2 <= n**2:
            if coloring[j + dU][i] == coloring[j - dU][i]:
                numMon[coloring[j + dU][i]] += 1
            dU += 1
        #Third term:
        dL = 1
        while (i - dL - n)**2 + (j - n)**2 <= n**2 and (i - 2*dL - n)**2 + (j - n)**2 <= n**2:
            if coloring[j][i - dL] == coloring[j][i - 2*dL]:
                numMon[coloring[j][i - dL]] += 1
            dL += 1
        dU = 1
        while (i - n)**2 + (j - dU - n)**2 <= n**2 and (i - n)**2 + (j - 2*dU - n)**2 <= n**2:
            if coloring[j - dU][i] == coloring[j - 2*dU][i]:
                numMon[coloring[j - dU][i]] += 1
            dU += 1

        

    min = numMon[0]
    minindex = 0
    index = 1

    while index < numColors:
        if numMon[index] < min:
            min = numMon[index]
            minindex = index
        index += 1

    if verbose == True:
        print(numMon)
    return minindex

##### Circle Optimization Code:

In [29]:
# Code that determines optimal coloring to minimize monochromatic triples in circle

def optimize_circle_coloring(coloring, progressiontype = 'D', ordering = None, verbose = False):

    if progressiontype not in ['P', 'D', 'UD']:
        print("The progression type is not an allowable type.  The choices are:")
        print("P: positive diagonals")
        print("D: all diagonals")
        print("UD: up/down and left/right progressions only")
        return

    # We'll assume ordering is a list of the numbers 1 through (2n + 1)^2.
    n = int(1.0*(len(coloring)-1)/2)
    w = 2*n + 1
    
    diditchange = 0

    currentcoloring = coloring

    if verbose:
        count = 0
        currentperformance = count_monochromatic_circle(currentcoloring, progressiontype=progressiontype)
        print("0: ", currentperformance)

    if ordering == None:
        ordering = range(w**2)
    
    while diditchange == 0:
        diditchange = 1
        for l in ordering:
            i = math.floor(l/w)
            j = l%w
            if (i-n)**2 + (j-n)**2 <= n**2:
                kcurrent = currentcoloring[i][j]
                knew = CheckSwap_Circle(currentcoloring, i, j, progressiontype=progressiontype)
                if kcurrent != knew:
                    diditchange = 0
                    currentcoloring[i][j] = knew

        if verbose:
            count += 1
            newperformance = count_monochromatic_circle(currentcoloring, progressiontype=progressiontype)
            print(count, ": ", newperformance)
            if newperformance > currentperformance:
                print("----")
                print("Error! This did not decrease the number of progressions.")
                print("----")
            currentperformance = newperformance

    if verbose:
        print("The total number of iterations (including the final check) was: ", count)

    print("Performance: ", count_monochromatic_circle(currentcoloring, progressiontype=progressiontype))
    plot_coloring(currentcoloring)
    
    return currentcoloring

##### Circle Plotting Code:

In [30]:
# Code to plot circle

# From ChatGPT
def plot_circle_coloring(Coloring, equal=True):
    rows, cols = len(Coloring), len(Coloring[0])
    fig, ax = plt.subplots()
    ax.set_xticks(np.arange(cols + 1), minor=False)
    ax.set_yticks(np.arange(rows + 1), minor=False)
    # ax.grid(which="minor", color="black", linestyle='-', linewidth=2)
    ax.tick_params(which="both", bottom=False, left=False, labelbottom=False, labelleft=False)
    
    for i in range(rows):
        for j in range(cols):
            color = 'blue' if Coloring[i][j] == 0 else 'red' if Coloring[i][j] == 1 else 'black'
            ax.add_patch(plt.Rectangle((j, i), 1, 1, color=color))
    
    ax.set_xlim(0, cols)
    ax.set_ylim(0, rows)
    if equal:
        ax.set_aspect('equal')
    plt.gca().invert_yaxis()
    plt.show()

### Unified Code:

In [16]:
# This function has the following parameters:

# First, the geometry can be:
# 'S' for kstar
# 'R' for rectangle
# 'C' for circle

# Second, each geometry can have specific progression types:
# For the kstar:
# 'adj' only allows for progressions to travel between adjacent pairs of arms
# 'all' allows for progressions to travel between all pairs of arms
# For the rectangle:
# 'D' if all diagonals are allowed
# 'P' if only positive diagonals are allowed
# 'UD' if only up/down or left/right progressions are allowed
# For the circle:
# 'D' if all diagonals are allowed
# 'P' if only positive diagonals are allowed


def optimize_coloring(geometry, progressiontype, coloring, ordering = None):
    if geometry == 'S' and progressiontype == 'adj':
        return optimize_adjacent_kstar_coloring(coloring, ordering)

    elif geometry == 'S' and progressiontype == 'all':
        return optimize_kstar_coloring(coloring, ordering)

    elif geometry == 'R': 
        return optimize_rectangle_coloring(coloring, progressiontype, ordering, aspect = True)

    else:
        return optimize_circle_coloring(coloring, progressiontype, ordering, verbose = False)

In [104]:
monochrome_k3_n500 = [[1]*500, [1]*500, [1]*500]
monochrome_k4_n500 = [[1]*500, [1]*500, [1]*500, [1]*500]
monochrome_k5_n500 = [[1]*500, [1]*500, [1]*500, [1]*500, [1]*500]
monochrome_k6_n500 = [[1]*500, [1]*500, [1]*500, [1]*500, [1]*500, [1]*500]
monochrome_k7_n500 = [[1]*500, [1]*500, [1]*500, [1]*500, [1]*500, [1]*500, [1]*500]
monochrome_k8_n500 = [[1]*500, [1]*500, [1]*500, [1]*500, [1]*500, [1]*500, [1]*500, [1]*500]

monochrome_k3_n1000 = [[1]*1000, [1]*1000, [1]*1000]
monochrome_k4_n1000 = [[1]*1000, [1]*1000, [1]*1000, [1]*1000]
monochrome_k5_n1000 = [[1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000]
monochrome_k6_n1000 = [[1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000]
monochrome_k7_n1000 = [[1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000]
monochrome_k8_n1000 = [[1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000, [1]*1000]

In [107]:
#Total possible monochromatic APs under all regime

print('all,', 'k = 3,', 'n = 500:', count_kstar(monochrome_k3_n500))
print('all,', 'k = 3,', 'n = 1000:', count_kstar(monochrome_k3_n1000))
print('all,', 'k = 4,', 'n = 500:', count_kstar(monochrome_k4_n500))
print('all,', 'k = 4,', 'n = 1000:', count_kstar(monochrome_k4_n1000))
print('all,', 'k = 5,', 'n = 500:', count_kstar(monochrome_k5_n500))
print('all,', 'k = 5,', 'n = 1000:', count_kstar(monochrome_k5_n1000))
print('all,', 'k = 6,', 'n = 500:', count_kstar(monochrome_k6_n500))
print('all,', 'k = 6,', 'n = 1000:', count_kstar(monochrome_k6_n1000))
print('all,', 'k = 7,', 'n = 500:', count_kstar(monochrome_k7_n500))
print('all,', 'k = 7,', 'n = 1000:', count_kstar(monochrome_k7_n1000))
print('all,', 'k = 8,', 'n = 500:', count_kstar(monochrome_k8_n500))
print('all,', 'k = 8,', 'n = 1000:', count_kstar(monochrome_k8_n1000))

all, k = 3, n = 500: 561750
all, k = 3, n = 1000: 2248500
all, k = 4, n = 500: 999000
all, k = 4, n = 1000: 3998000
all, k = 5, n = 500: 1561250
all, k = 5, n = 1000: 6247500
all, k = 6, n = 500: 2248500
all, k = 6, n = 1000: 8997000
all, k = 7, n = 500: 3060750
all, k = 7, n = 1000: 12246500
all, k = 8, n = 500: 3998000
all, k = 8, n = 1000: 15996000


In [111]:
#Total possible monochromatic APs under adjacent regime

print('adj,', 'k = 3,', 'n = 500:', count_adjacent_kstar(monochrome_k3_n500))
print('adj,', 'k = 3,', 'n = 1000:', count_adjacent_kstar(monochrome_k3_n1000))
print('adj,', 'k = 4,', 'n = 500:', count_adjacent_kstar(monochrome_k4_n500))
print('adj,', 'k = 4,', 'n = 1000:', count_adjacent_kstar(monochrome_k4_n1000))
print('adj,', 'k = 5,', 'n = 500:', count_adjacent_kstar(monochrome_k5_n500))
print('adj,', 'k = 5,', 'n = 1000:', count_adjacent_kstar(monochrome_k5_n1000))
print('adj,', 'k = 6,', 'n = 500:', count_adjacent_kstar(monochrome_k6_n500))
print('adj,', 'k = 6,', 'n = 1000:', count_adjacent_kstar(monochrome_k6_n1000))
print('adj,', 'k = 7,', 'n = 500:', count_adjacent_kstar(monochrome_k7_n500))
print('adj,', 'k = 7,', 'n = 1000:', count_adjacent_kstar(monochrome_k7_n1000))
print('adj,', 'k = 8,', 'n = 500:', count_adjacent_kstar(monochrome_k8_n500))
print('adj,', 'k = 8,', 'n = 1000:', count_adjacent_kstar(monochrome_k8_n1000))

adj, k = 3, n = 500: 561750
adj, k = 3, n = 1000: 2248500
adj, k = 4, n = 500: 749000
adj, k = 4, n = 1000: 2998000
adj, k = 5, n = 500: 936250
adj, k = 5, n = 1000: 3747500
adj, k = 6, n = 500: 1123500
adj, k = 6, n = 1000: 4497000
adj, k = 7, n = 500: 1310750
adj, k = 7, n = 1000: 5246500
adj, k = 8, n = 500: 1498000
adj, k = 8, n = 1000: 5996000
